In [9]:
import pandas as pd
import os
from unidecode import unidecode
import warnings
warnings.filterwarnings('ignore')

# Definir rutas de los archivos:

In [10]:
project_directory = os.path.abspath('../')
estadisticas_directory = os.path.join(project_directory, 'data', 'estadisticas')
estadisticas_files = os.listdir(estadisticas_directory)

# Exploración archivos:

## Reducción de columnas:

Para optimizar el uso del espacio en el bucket de data refinada en S3 y procesar solo la data necesaria en Lambda, se procede a identificar las columnas que no serán utilizadas.

### Definir campos comunes entre los archivos:

La lista resultante de campos será la primera base para definir los campos a guardar en el bucket de data refinada.

In [11]:
ls_raw_fields = []
for file in estadisticas_files:
    file_path = os.path.join(estadisticas_directory, file)
    df = pd.read_excel(file_path,nrows=0)
    ls_raw_fields_aux = df.columns.to_list()
    if len(ls_raw_fields)==0:
        ls_raw_fields = ls_raw_fields_aux
    else:
        list(set(ls_raw_fields).intersection(ls_raw_fields_aux))
ls_raw_fields

['MES',
 'COD_CONFIG_VEHICULO',
 'CONFIG_VEHICULO',
 'CODOPERACIONTRANSPORTE',
 'OPERACIONTRANSPORTE',
 'CODTIPOCONTENEDOR',
 'TIPOCONTENEDOR',
 'CODMUNICIPIOORIGEN',
 'MUNICIPIOORIGEN',
 'DEPARTAMENTOORIGEN',
 'CODMUNICIPIODESTINO',
 'MUNICIPIODESTINO',
 'DEPARTAMENTODESTINO',
 'CODMERCANCIA',
 'MERCANCIA',
 'NATURALEZACARGA',
 'VIAJESTOTALES',
 'KILOGRAMOS',
 'GALONES',
 'VIAJESLIQUIDOS',
 'VIAJESVALORCERO',
 'KILOMETROS',
 'VALORESPAGADOS',
 'CODMUNICIPIOINTERMEDIO',
 'MUNICIPIOINTERMEDIO',
 'DEPARTAMENTOINTERMEDIO',
 'KILOMETROSREGRESO',
 'KILOGRAMOSREGRESO',
 'GALONESREGRESO']

### Definir campos redundantes:

Se considera a los campos de esta lista como redundates porque ya se encuentran definidos por su correspondiente código.

In [12]:
ls_redundant_fields = ['OPERACIONTRANSPORTE','CONFIG_VEHICULO','TIPOCONTENEDOR', 'MUNICIPIOORIGEN', 'DEPARTAMENTOORIGEN', 'MUNICIPIODESTINO', 'DEPARTAMENTODESTINO', 
                        'MERCANCIA', 'MUNICIPIOINTERMEDIO', 'DEPARTAMENTOINTERMEDIO']

### Definir campos finales:

Estos serán los campos a consultar, explorar y guardar en el bucket de data refinada.

In [13]:
ls_refined_fields = [field for field in ls_raw_fields if field not in ls_redundant_fields]
print(len(ls_refined_fields))

19


## Identificar y definir tipos de datos:

El objetivo es identificar el tipo de datos presente en todos los archivos y saber si corresponden entre archivo y archivo. Estos serán el tipo de campos a asegurar durante el proceso de limpieza.

In [14]:
ls_raw_fields = []
df_info = pd.DataFrame()
for file in estadisticas_files:
    file_path = os.path.join(estadisticas_directory, file)
    df = pd.read_excel(file_path,usecols=ls_refined_fields)
    ls_columns = df.columns.tolist()
    ls_data_types = df.dtypes.tolist()
    df_info_aux = pd.DataFrame({'column_name': ls_columns, 'data_type': ls_data_types})
    if df_info.empty:
        df_info = df_info_aux
    else:
        df_info = pd.merge(df_info,df_info_aux,on = ['column_name','data_type'],how='left')

val =  len(df_info) == len(ls_refined_fields)
print(df_info)
print(val)

               column_name data_type
0                      MES     int64
1      COD_CONFIG_VEHICULO    object
2   CODOPERACIONTRANSPORTE    object
3        CODTIPOCONTENEDOR    object
4       CODMUNICIPIOORIGEN     int64
5      CODMUNICIPIODESTINO     int64
6             CODMERCANCIA    object
7          NATURALEZACARGA    object
8            VIAJESTOTALES     int64
9               KILOGRAMOS     int64
10                 GALONES     int64
11          VIAJESLIQUIDOS     int64
12         VIAJESVALORCERO     int64
13              KILOMETROS     int64
14          VALORESPAGADOS     int64
15  CODMUNICIPIOINTERMEDIO     int64
16       KILOMETROSREGRESO     int64
17       KILOGRAMOSREGRESO     int64
18          GALONESREGRESO     int64
True


### Se crea un diccionario con el tipo de datos:

Este diccionario se usará en el proceso de lectra de los archivos para asegurar el formato de los campos.

In [15]:
dict_types = df_info.set_index('column_name')['data_type'].to_dict()
dict_types

{'MES': dtype('int64'),
 'COD_CONFIG_VEHICULO': dtype('O'),
 'CODOPERACIONTRANSPORTE': dtype('O'),
 'CODTIPOCONTENEDOR': dtype('O'),
 'CODMUNICIPIOORIGEN': dtype('int64'),
 'CODMUNICIPIODESTINO': dtype('int64'),
 'CODMERCANCIA': dtype('O'),
 'NATURALEZACARGA': dtype('O'),
 'VIAJESTOTALES': dtype('int64'),
 'KILOGRAMOS': dtype('int64'),
 'GALONES': dtype('int64'),
 'VIAJESLIQUIDOS': dtype('int64'),
 'VIAJESVALORCERO': dtype('int64'),
 'KILOMETROS': dtype('int64'),
 'VALORESPAGADOS': dtype('int64'),
 'CODMUNICIPIOINTERMEDIO': dtype('int64'),
 'KILOMETROSREGRESO': dtype('int64'),
 'KILOGRAMOSREGRESO': dtype('int64'),
 'GALONESREGRESO': dtype('int64')}

## Validaciones:

En este paso se establecen las validaciones de los archivos, en caso de que algún archivo no supere la fase de validación no será cargado en el bucket de data refinada.

### Validar formato de archivos:

Se valida si el formato de los archivos

In [16]:
for file in estadisticas_files:
    if not file_path.endswith('.xlsx'):
        val = False
    else:
        val = True
    print(f'file: {file} | validation: {val}')

file: EstadisticasRNDC_202207.xlsx | validation: True
file: EstadisticasRNDC_202208.xlsx | validation: True
file: EstadisticasRNDC_202209.xlsx | validation: True
file: EstadisticasRNDC_202210.xlsx | validation: True
file: EstadisticasRNDC_202211.xlsx | validation: True
file: EstadisticasRNDC_202212.xlsx | validation: True
file: EstadisticasRNDC_202301.xlsx | validation: True
file: EstadisticasRNDC_202302.xlsx | validation: True
file: EstadisticasRNDC_202303.xlsx | validation: True
file: EstadisticasRNDC_202304.xlsx | validation: True


###  Validar campos:

Se valida si todos los campos definidos están presentes en el archivo.

In [17]:
for file in estadisticas_files:
    file_path = os.path.join(estadisticas_directory, file)
    df = pd.read_excel(file_path,nrows=0)
    val = all(elem in df.columns.to_list() for elem in ls_refined_fields)
    print(f'file {file} | validation: {val}')

file EstadisticasRNDC_202207.xlsx | validation: True
file EstadisticasRNDC_202208.xlsx | validation: True
file EstadisticasRNDC_202209.xlsx | validation: True
file EstadisticasRNDC_202210.xlsx | validation: True
file EstadisticasRNDC_202211.xlsx | validation: True
file EstadisticasRNDC_202212.xlsx | validation: True
file EstadisticasRNDC_202301.xlsx | validation: True
file EstadisticasRNDC_202302.xlsx | validation: True
file EstadisticasRNDC_202303.xlsx | validation: True
file EstadisticasRNDC_202304.xlsx | validation: True


### Validar coherencia en los datos:

Se valida si el periodo al que hace referencia el nombre del archivo corresponde con el dato de la columna "MES". Para el momento de la limpieza si el archivo no cumple esta validación no será procesado ni cargado en el bucked de data refinada.

In [18]:
for file in estadisticas_files:
    file_path = os.path.join(estadisticas_directory, file)
    period = float(file.split('_')[-1].split('.')[0])
    df = pd.read_excel(file_path,usecols=['MES'])
    period_avg = df.MES.mean()
    val = period == period_avg
    print(f'file: {file} | period: {period} | period_avg: {period_avg} | validation: {val}')

file: EstadisticasRNDC_202207.xlsx | period: 202207.0 | period_avg: 202207.0 | validation: True
file: EstadisticasRNDC_202208.xlsx | period: 202208.0 | period_avg: 202208.0 | validation: True
file: EstadisticasRNDC_202209.xlsx | period: 202209.0 | period_avg: 202209.0 | validation: True
file: EstadisticasRNDC_202210.xlsx | period: 202210.0 | period_avg: 202210.0 | validation: True
file: EstadisticasRNDC_202211.xlsx | period: 202211.0 | period_avg: 202211.0 | validation: True
file: EstadisticasRNDC_202212.xlsx | period: 202212.0 | period_avg: 202212.0 | validation: True
file: EstadisticasRNDC_202301.xlsx | period: 202301.0 | period_avg: 202302.0 | validation: False
file: EstadisticasRNDC_202302.xlsx | period: 202302.0 | period_avg: 202302.0 | validation: True
file: EstadisticasRNDC_202303.xlsx | period: 202303.0 | period_avg: 202303.0 | validation: True
file: EstadisticasRNDC_202304.xlsx | period: 202304.0 | period_avg: 202304.0 | validation: True


# Transformaciones y limpieza:

En este paso entrarán todos los archivos que hayan pasado la fase de validación. Sin embargo, para el ejemplo de este notebook ssolo se usará el archivo "EstadisticasRNDC_202207.xlsx"

In [19]:
file = 'EstadisticasRNDC_202207.xlsx'
file_path = os.path.join(estadisticas_directory, file)
df = pd.read_excel(file_path,usecols=ls_refined_fields,dtype=dict_types)
df.rename(columns={'MES':'ANOMES'},inplace=True)
df.drop_duplicates(inplace=True) # eliminar duplicados
df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x) # volver mayúsculas los campos str
df = df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x) # quitar acentos a campos str
print(f'file: {file} | status: refined')

file: EstadisticasRNDC_202207.xlsx | status: refined


In [20]:
df.head()

,ANOMES,COD_CONFIG_VEHICULO,CODOPERACIONTRANSPORTE,CODTIPOCONTENEDOR,CODMUNICIPIOORIGEN,CODMUNICIPIODESTINO,CODMERCANCIA,NATURALEZACARGA,VIAJESTOTALES,KILOGRAMOS,GALONES,VIAJESLIQUIDOS,VIAJESVALORCERO,KILOMETROS,VALORESPAGADOS,CODMUNICIPIOINTERMEDIO,KILOMETROSREGRESO,KILOGRAMOSREGRESO,GALONESREGRESO
0,202207,CA,G,.,5001000,5686000,009980,CARGA NORMAL,2,1200,0,0,1,77,400000,0,0,0,0
1,202207,CA,G,.,5001000,5607000,008508,CARGA NORMAL,1,2000,0,0,0,31,300000,0,0,0,0
2,202207,CA,G,.,5001000,50001000,009403,CARGA NORMAL,1,1500,0,0,0,531,900000,0,0,0,0
3,202207,CA,G,.,5001000,23855000,006501,CARGA NORMAL,1,70,0,0,0,462,80000,0,0,0,0
4,202207,CA,G,.,5001000,5400000,009880,CARGA NORMAL,2,1936,0,0,0,58,436924,0,0,0,0
